In [1]:
import torch
import argparse
import numpy as np
import torch.distributed as dist
import os
from PIL import Image
from tqdm.auto import tqdm
import json
from dataclasses import dataclass


from relighting.inpainter import BallInpainter

from relighting.mask_utils import MaskGenerator
from relighting.ball_processor import (
    get_ideal_normal_ball,
    crop_ball
)
from relighting.dataset import GeneralLoader
from relighting.utils import name2hash
from relighting.environment_map_projector import EnvironmentMapProjector
from relighting.environment_map_projector import EnvironmentMapProjector
from relighting.exposure_bracketer import ExposureBracketer
from relighting.chrome_ball_generator import ChromeBallGenerator
import relighting.dist_utils as dist_util
import time


/home/coder/work/DiffusionLight/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chromeBallGenerator = ChromeBallGenerator()
chromeBallGenerator.load_model()

/home/coder/work/DiffusionLight/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  5.92it/s]


Disabled watermasking
using lora path models/ThisIsTheFinal-lora-hdr-continuous-largeT@900/0_-5/checkpoint-2500
using lora scale 0.75
compiling unet model
Model compilation time:  0.004719972610473633
Loaded model!


In [4]:
environmentMapProjector = EnvironmentMapProjector()
image = Image.open("example/bed.png")
cropped_spheres = chromeBallGenerator.generate_spheres(image, 30)
env_maps = {}
for ev, im in cropped_spheres.items():
    im.save(f"output/{ev}.png")
    env_maps[ev] = environmentMapProjector(im) / 255
    Image.fromarray((env_maps[ev] * 255).astype(np.uint8)).save(f"output/bed_ev-{int(ev*10)}_map.png")

interpolate embedding...
EV :  [0.0, -5.0]
EV :  [-0.0, 1.0]


/home/coder/work/DiffusionLight/.venv/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/coder/work/DiffusionLight/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Ran model!
Ran model!


In [25]:
import importlib
import relighting
import relighting.exposure_bracketer
import os
import cv2

import relighting.exposure_bracketer
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
importlib.reload(relighting.exposure_bracketer)

exposureBracketer = relighting.exposure_bracketer.ExposureBracketer()
hdr = exposureBracketer(env_maps)
cv2.imwrite("output/res.exr", hdr.astype(np.float32))

True